In [7]:
import firebase_admin
from pathlib import Path
from firebase_admin import credentials
from firebase_admin import firestore
import warnings
import os
warnings.filterwarnings('ignore')

In [8]:
# Use the path to your service account key file
PARENT_DIR = Path('__file__').resolve().parent
print(PARENT_DIR)


C:\Users\encaa\OneDrive\Desktop\Mac_Geo\Geo_location\xcor


In [ ]:
cred = credentials.Certificate(os.path.join(PARENT_DIR,"serviceAccountKey.json"))

# Initialize the app with your project's credentials
# You can also specify the database URL if it's not the default
firebase_admin.initialize_app(cred, {
    'projectId': 'geolocation-app-b549e',
})

# Now you can use the Firebase services
# For example, to access Firestore:
db = firestore.client()

In [10]:
def get_all_sessions():
    """Retrieve all session data from Firestore"""
    if not db:
        print("Firebase not initialized. Please check your configuration.")
        return None
    
    try:
        # Get all documents from the 'locations' collection
        locations_ref = db.collection('sessions')
        docs = locations_ref.stream()
        
        locations_data = []
        for doc in docs:
            data = doc.to_dict()
            data['document_id'] = doc.id  # Add document ID for reference
            locations_data.append(data)
        
        print(f"Retrieved {len(locations_data)} location records")
        return locations_data
    except Exception as e:
        print(f"Error retrieving data: {e}")
        return None

In [11]:
import math

def haversine_distance(lat1, lon1, lat2, lon2, radius=6371):
    """
    Calculates the Haversine distance between two points on Earth.

    Args:
        lat1 (float): Latitude of the first point in degrees.
        lon1 (float): Longitude of the first point in degrees.
        lat2 (float): Latitude of the second point in degrees.
        lon2 (float): Longitude of the second point in degrees.
        radius (float): Radius of the Earth in kilometers (default is 6371 km).

    Returns:
        float: The distance between the two points in kilometers.
    """
    # Convert latitudes and longitudes from degrees to radians
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)

    # Calculate the difference in latitudes and longitudes
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad

    # Haversine formula
    a = math.sin(dlat / 2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    distance = radius * c * 1000
    return distance



In [13]:
sessions_data = get_all_sessions()

if sessions_data:
    print(f"\nSuccessfully connected! Found {len(sessions_data)} location records.")
    print("\nSample data structure:")
    if sessions_data:
        sample = sessions_data[0]
        for key, value in sample.items():
            print(f"  {key}: {value}")
else:
    print("Failed to retrieve data. Please check your Firebase configuration.")


Retrieved 3 location records

Successfully connected! Found 3 location records.

Sample data structure:
  createdAt: 2025-09-24 23:58:21.735000+00:00
  startTime: 2025-09-25T00:00:00.000Z
  events: [{'location': {'accuracy': 23.11442339027511, 'longitude': -90.464876955105, 'latitude': 30.51039841855592, 'timestamp': '2025-09-24T23:58:03.528Z'}, 'type': 'created', 'by': 'a2', 'timestamp': '2025-09-24T23:58:21.332Z'}, {'location': {'accuracy': 24.50821576939994, 'longitude': -90.46457886676608, 'latitude': 30.510607101514616, 'timestamp': '2025-09-24T23:58:06.383Z'}, 'type': 'joined', 'by': 'e2', 'timestamp': '2025-09-24T23:58:38.926Z'}, {'audioUrl': 'https://firebasestorage.googleapis.com/v0/b/geolocation-app-b549e.firebasestorage.app/o/audio%2Fchurch_e2_2025-09-25_19%3A00.webm?alt=media&token=7a7849a6-d2fa-4072-b6e5-1d7f051d5803', 'type': 'recording_uploaded', 'duration': 15, 'by': 'e2', 'timestamp': '2025-09-25T00:00:26.105Z'}, {'audioUrl': 'https://firebasestorage.googleapis.com/v0/

In [14]:
sessions_data

[{'createdAt': DatetimeWithNanoseconds(2025, 9, 24, 23, 58, 21, 735000, tzinfo=datetime.timezone.utc),
  'startTime': '2025-09-25T00:00:00.000Z',
  'events': [{'location': {'accuracy': 23.11442339027511,
     'longitude': -90.464876955105,
     'latitude': 30.51039841855592,
     'timestamp': '2025-09-24T23:58:03.528Z'},
    'type': 'created',
    'by': 'a2',
    'timestamp': '2025-09-24T23:58:21.332Z'},
   {'location': {'accuracy': 24.50821576939994,
     'longitude': -90.46457886676608,
     'latitude': 30.510607101514616,
     'timestamp': '2025-09-24T23:58:06.383Z'},
    'type': 'joined',
    'by': 'e2',
    'timestamp': '2025-09-24T23:58:38.926Z'},
   {'audioUrl': 'https://firebasestorage.googleapis.com/v0/b/geolocation-app-b549e.firebasestorage.app/o/audio%2Fchurch_e2_2025-09-25_19%3A00.webm?alt=media&token=7a7849a6-d2fa-4072-b6e5-1d7f051d5803',
    'type': 'recording_uploaded',
    'duration': 15,
    'by': 'e2',
    'timestamp': '2025-09-25T00:00:26.105Z'},
   {'audioUrl': 'htt

In [15]:
sessions_data[0]

{'createdAt': DatetimeWithNanoseconds(2025, 9, 24, 23, 58, 21, 735000, tzinfo=datetime.timezone.utc),
 'startTime': '2025-09-25T00:00:00.000Z',
 'events': [{'location': {'accuracy': 23.11442339027511,
    'longitude': -90.464876955105,
    'latitude': 30.51039841855592,
    'timestamp': '2025-09-24T23:58:03.528Z'},
   'type': 'created',
   'by': 'a2',
   'timestamp': '2025-09-24T23:58:21.332Z'},
  {'location': {'accuracy': 24.50821576939994,
    'longitude': -90.46457886676608,
    'latitude': 30.510607101514616,
    'timestamp': '2025-09-24T23:58:06.383Z'},
   'type': 'joined',
   'by': 'e2',
   'timestamp': '2025-09-24T23:58:38.926Z'},
  {'audioUrl': 'https://firebasestorage.googleapis.com/v0/b/geolocation-app-b549e.firebasestorage.app/o/audio%2Fchurch_e2_2025-09-25_19%3A00.webm?alt=media&token=7a7849a6-d2fa-4072-b6e5-1d7f051d5803',
   'type': 'recording_uploaded',
   'duration': 15,
   'by': 'e2',
   'timestamp': '2025-09-25T00:00:26.105Z'},
  {'audioUrl': 'https://firebasestorage.g

In [16]:
# lat_s, lon_s = locations_data[0]['latitude'], locations_data[0]['longitude']
# lat_e, lon_e = locations_data[2]['latitude'], locations_data[2]['longitude']

# distance = haversine_distance(lat_s, lon_s, lat_e, lon_e)
# print(f"Distance between {locations_data[0]['username']} and {locations_data[2]['username']}: {distance:.2f} m")


In [17]:
sessions_data[0]['participants'][0]['audioUrl']

'https://firebasestorage.googleapis.com/v0/b/geolocation-app-b549e.firebasestorage.app/o/audio%2Fchurch_a2_2025-09-25_19%3A00.webm?alt=media&token=19f9cdd6-3549-4469-a930-71b09f923e4a'

In [18]:
import librosa
import io

In [21]:
'''
import requests
par_session = sessions_data[2]['participants'] 
for i in range(len(par_session)):
    try:
        print(i)
        audio = par_session[i]['audioUrl']
        response = requests.get(audio)
        print('----')
        audio_data = response.content
        print(audio_data)
        audio_io = io.BytesIO(audio_data)
        print('===')
        y, sr = librosa.load(audio_io, sr=None)
        print('+++')
        print(y, sr)
        break
        # with open(f'audio_{par_session[i]["username"]}.webm', 'wb') as f:
        #     f.write(response.content)
    except Exception as e:
        print(f"Error downloading audio for {par_session[i]['username']}: {e}")
'''


'\nimport requests\npar_session = sessions_data[2][\'participants\'] \nfor i in range(len(par_session)):\n    try:\n        print(i)\n        audio = par_session[i][\'audioUrl\']\n        response = requests.get(audio)\n        print(\'----\')\n        audio_data = response.content\n        print(audio_data)\n        audio_io = io.BytesIO(audio_data)\n        print(\'===\')\n        y, sr = librosa.load(audio_io, sr=None)\n        print(\'+++\')\n        print(y, sr)\n        break\n        # with open(f\'audio_{par_session[i]["username"]}.webm\', \'wb\') as f:\n        #     f.write(response.content)\n    except Exception as e:\n        print(f"Error downloading audio for {par_session[i][\'username\']}: {e}")\n'

# 